In [1]:
import numpy as np
import tensorflow as tf
from sklearn import datasets
from sklearn.preprocessing import minmax_scale, LabelBinarizer
from keras.layers import Dense
from keras import backend as K

Using TensorFlow backend.


In [2]:
ds = datasets.load_iris()
X_train, y_train = minmax_scale(ds.data), LabelBinarizer().fit_transform(ds.target)

In [3]:
n_X_samples = X_train.shape[0]
n_X_features = X_train.shape[1]
n_Z_features = 6

In [4]:
X = tf.placeholder(tf.float32, shape=[None, n_X_features], name='X')
D_W1 = tf.Variable(tf.random_uniform([n_X_features, 6]), name='D_W1')
D_b1 = tf.Variable(tf.random_uniform([6]), name='D_b1')
D_W2 = tf.Variable(tf.random_uniform([6, 1]), name='D_W2')
D_b2 = tf.Variable(tf.random_uniform([1]), name='D_b2')
D_parameters = [D_W1, D_W2, D_b1, D_b2]
def D_logit(X):
    D_h1 = tf.nn.tanh(tf.matmul(X, D_W1) + D_b1)
    return tf.matmul(D_h1, D_W2) + D_b2

Z = tf.placeholder(tf.float32, shape=[None, n_Z_features], name='Z')
G_W1 = tf.Variable(tf.random_uniform([n_Z_features, 6]), name='G_W1')
G_b1 = tf.Variable(tf.random_uniform([6]), name='G_b1')
G_W2 = tf.Variable(tf.random_uniform([6, n_X_features]), name='G_W2')
G_b2 = tf.Variable(tf.random_uniform([n_X_features]), name='G_b2')
G_parameters = [G_W1, G_W2, G_b1, G_b2]
def G_logit(Z):
    G_h1 = tf.nn.tanh(tf.matmul(Z, G_W1) + G_b1)
    return tf.matmul(G_h1, G_W2) + G_b2

def sample_Z(n_samples, n_features):
    return np.random.uniform(-1., 1., size=[n_samples, n_features]).astype(np.float32)

In [5]:
D_logit_data = D_logit(X)
D_logit_generated = D_logit(tf.nn.sigmoid(G_logit(Z)))

D_loss_data = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_data, labels=tf.ones_like(D_logit_data)))
D_loss_generated = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_generated, labels=tf.zeros_like(D_logit_generated)))
D_loss = D_loss_data + D_loss_generated

G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_generated, labels=tf.ones_like(D_logit_generated)))

In [6]:
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=D_parameters)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=G_parameters)

In [7]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

batch_size = 10
n_batches = int(n_X_samples / batch_size)
for epoch in range(50):
    X_epoch = X_train[np.random.permutation(range(n_X_samples))]
    for batch_index in range(n_batches):
        start_index = batch_index * batch_size
        end_index = start_index + batch_size
        X_batch = X_epoch[start_index:end_index]
        _, D_loss_value = sess.run([D_solver, D_loss], feed_dict={X: X_batch, Z: sample_Z(batch_size, n_Z_features)})
        _, G_loss_value = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(batch_size, n_Z_features)})
    print('Epoch: {}, discriminator loss: {}, generator loss: {}'.format(epoch, D_loss_value, G_loss_value))

Epoch: 0, discriminator loss: 3.394270181655884, generator loss: 0.03680562227964401
Epoch: 1, discriminator loss: 3.3111073970794678, generator loss: 0.0431181862950325
Epoch: 2, discriminator loss: 3.1722819805145264, generator loss: 0.04437081143260002
Epoch: 3, discriminator loss: 3.118732213973999, generator loss: 0.050017885863780975
Epoch: 4, discriminator loss: 2.9895150661468506, generator loss: 0.05530524253845215
Epoch: 5, discriminator loss: 2.8761837482452393, generator loss: 0.06776635348796844
Epoch: 6, discriminator loss: 2.7620742321014404, generator loss: 0.06646376848220825
Epoch: 7, discriminator loss: 2.755126714706421, generator loss: 0.07918612658977509
Epoch: 8, discriminator loss: 2.616192102432251, generator loss: 0.08883442729711533
Epoch: 9, discriminator loss: 2.551145315170288, generator loss: 0.09537789970636368
Epoch: 10, discriminator loss: 2.4858713150024414, generator loss: 0.10494150966405869
Epoch: 11, discriminator loss: 2.398826837539673, generato

In [8]:
X_generated = tf.nn.sigmoid(G_logit(sample_Z(10, n_Z_features)))

In [ ]:
tf.nn.tanh(tf.matmul(Z, G_W1) + G_b1)

In [11]:
tf.nn.sigmoid(G_logit(sample_Z(10, n_Z_features)))

<tf.Tensor 'Sigmoid_1:0' shape=(10, 4) dtype=float32>

In [12]:
sess.run(G_W1)

array([[ 0.22884381, -0.01428368, -0.01331964,  0.60230684,  0.50267297,
         0.1466855 ],
       [ 0.41315621,  0.05289599,  0.48941782,  0.62571698,  0.41351119,
         0.54372603],
       [ 0.31254059,  0.1366342 ,  0.1802099 , -0.08998414, -0.03047688,
         0.20382974],
       [ 0.3270953 ,  0.07598057,  0.00336073,  0.00547926,  1.09153223,
         0.12950367],
       [ 0.30255401,  0.08746911, -0.01836257, -0.1077238 ,  0.80138707,
         0.47222441],
       [ 0.04228994,  0.19683203,  0.39572161,  0.51092356,  0.13156611,
         0.08996762]], dtype=float32)